In [17]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import os
from scipy.optimize import curve_fit

## Data preparation

### Load data

In [18]:
chirp_direction = 'chirp_up'
data_file_path = r'C:\Users\vaish\dsvv_atomionics\compiled_data\compiled_' + f'{chirp_direction}.pkl'
data_cols = [
		'chirp_rate', 'fraction',
		'CA+_mean', 'CA+_std_dev', 'CA+_percentile_0', 'CA+_percentile_10', 'CA+_percentile_20', 
		'CA+_percentile_30', 'CA+_percentile_40', 'CA+_percentile_50', 'CA+_percentile_60', 'CA+_percentile_70', 
		'CA+_percentile_80', 'CA+_percentile_90', 'CA+_percentile_100', 'CA-_mean', 'CA-_std_dev', 'CA-_percentile_0', 
		'CA-_percentile_10', 'CA-_percentile_20', 'CA-_percentile_30', 'CA-_percentile_40', 'CA-_percentile_50', 
		'CA-_percentile_60', 'CA-_percentile_70', 'CA-_percentile_80', 'CA-_percentile_90', 'CA-_percentile_100'
	]
data = pd.DataFrame(pd.read_pickle(data_file_path), columns=data_cols)

# 80-20 train/test split
train_data_df = data.sample(frac=0.8, random_state=0)
test_data_df = data.drop(train_data_df.index)

print(train_data_df.shape)
# print(train_data.head())
print(test_data_df.shape)
# print(test_data.head())

(253, 28)
(63, 28)


### Constant parameters

In [19]:
# I'm not sure of the calculations here, copied from the original code
# constants for evaluation: keff, bigT, chirp_direction
speed_of_light_in_vacuum = 299792458
f1 = 384.2304844685e12 - 72.9113e6 - 2.56300597908911e9 + 61.625e6
f2 = f1 + 6.725e9 + 112.06936e6
k1 = 2*np.pi/(speed_of_light_in_vacuum/f1)
k2 = 2*np.pi/(speed_of_light_in_vacuum/f2)
Keff = k1+k2

bigT = 10e-3

## Custom Loss Function

In [20]:
def sine(alpha, contrast, acc_due_to_gravity, fringe_offset):
    phi = (Keff * acc_due_to_gravity - 2 * np.pi * alpha) * (bigT**2)
    return (-contrast * np.cos(phi) + fringe_offset)

In [21]:
def get_popt_for_best_fit(corrected_chirp_rates: np.ndarray, observed_fractions: np.ndarray):
	contrast = (np.max(observed_fractions) - np.min(observed_fractions)) / 2
	fringe_offset = np.mean(observed_fractions)
	g_guess = -9.78306 if chirp_direction == 'chirp_down' else 9.77997
	period = 780e-9/(2*bigT**2)

	sine_function_parameters = [contrast, g_guess, fringe_offset]
	bounds = ([0, g_guess - 0.501*period, -np.inf], [1, g_guess + 0.501*period, np.inf])
	popt, _ = curve_fit(sine, corrected_chirp_rates, observed_fractions, p0=sine_function_parameters, maxfev=1000000, bounds=bounds)
	return popt

In [22]:
def calculate_mse_loss_with_popt(outputs: torch.TensorType, observed_fractions: torch.TensorType, *popt):
	print(popt)
	tensor0 = torch.tensor([Keff * popt[1] * bigT**2 for i in range(len(outputs))], dtype=torch.float32)
	tensor1 = outputs * bigT**2 * 2 * torch.pi
	phi_tensor = torch.cos(tensor0 - tensor1)
	predicted_fractions_tensor = phi_tensor * (-popt[0]) + popt[2]
	mse_loss = nn.MSELoss()
	loss = mse_loss(predicted_fractions_tensor, observed_fractions)
	return loss

## Benchmark MSE score

In [23]:
test_chirp_rates = test_data_df['chirp_rate'].to_numpy()
test_fractions = test_data_df['fraction'].to_numpy()
popt = get_popt_for_best_fit(test_chirp_rates, test_fractions)

residuals = test_fractions - sine(test_chirp_rates, *popt)
mse = np.mean(residuals**2)
print(f'MSE: {mse}')

MSE: 0.0075971372770962265


## Training

### Neural Network

In [24]:
class TransformationNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TransformationNN, self).__init__()
        # Define the layers of the network: 3 fully connected layers 27 -> 64 -> 64 -> 1
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_dim)
    
    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

In [25]:
training_data_tensor = torch.tensor(train_data_df.drop(columns=['fraction']).values, dtype=torch.float32)
observed_fractions_tensor = torch.tensor(train_data_df['fraction'].values, dtype=torch.float32)
observed_fractions_ndarray = train_data_df['fraction'].to_numpy()

In [26]:
# Initialize the neural network
model = TransformationNN(input_dim=27, output_dim=1)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [28]:

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    
    # Forward pass: Compute predicted y by passing x to the model
    outputs = model(training_data_tensor)
    
    outputs_clone = outputs.clone()
    corrected_chirp_rates_ndarray = outputs_clone.detach().cpu().squeeze().numpy()
    
    popt = get_popt_for_best_fit(corrected_chirp_rates_ndarray, observed_fractions_ndarray)    
    loss = calculate_mse_loss_with_popt(outputs, observed_fractions_tensor, *popt)
    
    # Print the loss
    print(f'Epoch [{epoch + 1}/{num_epochs}], MSE: {loss.item():.4f}')

    # Zero gradients, perform a backward pass, and update the weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

(0.9999996966402528, 9.781279726769677, 0.5458904276171672)
Epoch [1/50], MSE: 0.0069
(0.9999388693243132, 9.781275045614086, 0.5534109281859813)
Epoch [2/50], MSE: 0.0069
(0.12839251028619958, 9.77996931414185, 0.6573773917927895)
Epoch [3/50], MSE: 0.0069
(0.9999539207321785, 9.77932933194087, 0.548908018320777)
Epoch [4/50], MSE: 0.0069
(0.999988178460198, 9.779331141240263, 0.5518034200075803)
Epoch [5/50], MSE: 0.0069
(0.9999733937114497, 9.779334117873312, 0.5565178668864196)
Epoch [6/50], MSE: 0.0069
(0.9999886876016091, 9.779328421670035, 0.5473310462361387)
Epoch [7/50], MSE: 0.0069
(0.9999949578287651, 9.779329439103337, 0.5490676093637435)
Epoch [8/50], MSE: 0.0069
(0.999991397384715, 9.779331088865192, 0.5517836231878152)
Epoch [9/50], MSE: 0.0069
(0.9999985723448706, 9.779329672934422, 0.5494975661615097)
Epoch [10/50], MSE: 0.0069
(0.9999998923769207, 9.77932862839999, 0.5477239322601127)
Epoch [11/50], MSE: 0.0069
(0.9999455905251193, 9.779334500012588, 0.557042473696033

### Genetic Algorithm